## NER - Natasha

Natasha  - питоновская библиотека для извлечения именованных сущностей. Она похоже на Tomita-parser, но в ней все на чистом питоне, с открытым кодом и активно развивается. 

Если быть точнее, то natasha - набор готовых правил для парсера yargy. 

Есть например готовые правила для извлечения персон.

In [ ]:
# Установить можно через pip.
from natasha import NamesExtractor

In [ ]:
sents = open('sents.txt').read().splitlines()

In [ ]:
extractor = NamesExtractor()

In [ ]:
extractor('\n'.join(sents))

Видно, что извлекается не очень хорошо. К тому же в реальной задаче нам потребуется извлекать ещё какие-то аттрибуты сущностей и группировать их в факты. И не обязательно этих типов. Поэтому полезно разобраться с самим парсером yargy.

Документация yargy: https://yargy.readthedocs.io/ru/latest/reference.html

Лучше всего с такими штуками разбираться на практике. Давайте попробуем написать правила для извлечения персон. Каждая персона должна описываться 3 полями - Имя, Фамилия, Отчество. Также у Персоны должны быть атрибуты - должность и место работы. 

За основу возьмем пример из документации:

In [ ]:
from yargy import Parser
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display

Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)


POSITION = morph_pipeline([
    'премьер министр',
    'президент'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

Посмотрим, что получается:

In [ ]:
for sent in sents[:100]:
    print(sent)
    for match in parser.findall(sent):
        display(match.fact)
    print('---------------')

Чтобы проверить какие морфологические тэги приписываются словам, можно использовать MorphTokenizer.

In [ ]:
tokenizer = MorphTokenizer()

In [ ]:
list(tokenizer('Медведев'))

## Задача на семинар:
Доработать правила так, чтобы извлекалось как можно больше правильных фактов.